In [144]:
import numpy as np
from numpy import asarray
from PIL import Image
import pandas as pd
import tqdm
import os
import glob
import torch
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout,AdaptiveAvgPool2d
from torch.optim import Adam, SGD

In [60]:
dir = '/Users/miaoxinran/Documents/WISC/imagenet/Images/*'

In [153]:
folders = glob.glob(dir)

Image_paths = []
Labels = []

for folder in folders:
    for f in glob.glob(folder+'/*'):
        Image_paths.append(f)
        Labels.append(folder.split('/')[7])
        
sub_imagenet = {'image_paths':Image_paths,'Wnids':Labels}
sub_imagenet = pd.DataFrame(sub_imagenet)

In [154]:
sub_imagenet.head()

,image_paths,Wnids
0,/Users/miaoxinran/Documents/WISC/imagenet/Imag...,n02336641
1,/Users/miaoxinran/Documents/WISC/imagenet/Imag...,n02336641
2,/Users/miaoxinran/Documents/WISC/imagenet/Imag...,n02336641
3,/Users/miaoxinran/Documents/WISC/imagenet/Imag...,n02336641
4,/Users/miaoxinran/Documents/WISC/imagenet/Imag...,n02336641


In [ ]:
category = sub_imagenet.Wnids.astype('category').cat.codes

In [126]:
def add_noise(img_arr,mean=0,std=1):
    noisy_img = img_arr + np.random.normal(mean, std, img_arr.shape)
    noisy_img_clipped = np.clip(noisy_img, 0, 255)
    return noisy_img_clipped

In [347]:
def preprocess(img_path):
    img_rz = np.array(Image.open(img_path).resize((256,256)))
    if len(img_rz.shape) == 2:
        img_rz = img_rz.reshape(256,256,1)
        img_13 = np.dstack((add_noise(img_rz), add_noise(img_rz), add_noise(img_rz), add_noise(img_rz),add_noise(img_rz),
                           add_noise(img_rz), add_noise(img_rz), add_noise(img_rz), add_noise(img_rz),add_noise(img_rz),
                           add_noise(img_rz), add_noise(img_rz),add_noise(img_rz)))
    elif img_rz.shape[2] == 3:
        img_13 = np.dstack((add_noise(img_rz),
                        add_noise(img_rz),
                       add_noise(img_rz),
                       add_noise(img_rz),
                       add_noise(img_rz[:,:,0])))
    elif img_rz.shape[2] == 4:
        img_13 = np.dstack((add_noise(img_rz),
                        add_noise(img_rz),
                       add_noise(img_rz),
                       add_noise(img_rz[:,:,0])))
    img_13 = img_13.astype(np.uint8)
    return img_13

In [350]:
Img = []
for img_path in tqdm.tqdm(sub_imagenet['image_paths']):
    try:
        img = preprocess(img_path)
        if img.shape != (256,256,13):
            print(img_path,img.shape)
        Img.append(img)
    except PIL.UnidentifiedImageError:
        Emtpy_img_path.append(img_path)
    except IndexError:
        print(img_path,np.array(Image.open(img_path)).shape)
        Emtpy_img_path.append(img_path)



  0%|          | 0/13931 [00:00<?, ?it/s]

  0%|          | 2/13931 [00:00<11:53, 19.52it/s]

  0%|          | 4/13931 [00:00<11:59, 19.37it/s]

  0%|          | 7/13931 [00:00<10:56, 21.20it/s]

  0%|          | 9/13931 [00:00<11:14, 20.63it/s]

  0%|          | 11/13931 [00:00<11:41, 19.86it/s]

  0%|          | 15/13931 [00:00<10:05, 22.99it/s]

  0%|          | 18/13931 [00:00<10:51, 21.37it/s]

  0%|          | 21/13931 [00:00<11:28, 20.20it/s]

  0%|          | 24/13931 [00:01<11:33, 20.06it/s]

  0%|          | 27/13931 [00:01<10:30, 22.06it/s]

  0%|          | 30/13931 [00:01<09:52, 23.45it/s]

  0%|          | 33/13931 [00:01<09:15, 25.00it/s]

  0%|          | 37/13931 [00:01<08:59, 25.76it/s]

  0%|          | 41/13931 [00:01<08:06, 28.55it/s]

  0%|          | 44/13931 [00:01<08:02, 28.77it/s]

  0%|          | 47/13931 [00:01<08:08, 28.45it/s]

  0%|          | 50/13931 [00:02<08:04, 28.62it/s]

  0%|          | 53/13931 [00:02<09:26, 24.49it/s]

  0%|          | 57/139

  6%|▌         | 795/13931 [00:36<11:47, 18.58it/s]

  6%|▌         | 797/13931 [00:36<11:44, 18.65it/s]

  6%|▌         | 799/13931 [00:36<11:31, 18.98it/s]

  6%|▌         | 801/13931 [00:37<11:40, 18.74it/s]

  6%|▌         | 803/13931 [00:37<11:36, 18.86it/s]

  6%|▌         | 806/13931 [00:37<11:18, 19.33it/s]

  6%|▌         | 808/13931 [00:37<11:21, 19.25it/s]

  6%|▌         | 811/13931 [00:37<10:27, 20.91it/s]

  6%|▌         | 814/13931 [00:37<10:39, 20.52it/s]

  6%|▌         | 817/13931 [00:37<10:49, 20.19it/s]

  6%|▌         | 820/13931 [00:38<11:03, 19.78it/s]

  6%|▌         | 822/13931 [00:38<11:06, 19.66it/s]

  6%|▌         | 824/13931 [00:38<11:24, 19.14it/s]

  6%|▌         | 827/13931 [00:38<10:24, 20.99it/s]

  6%|▌         | 830/13931 [00:38<10:36, 20.59it/s]

  6%|▌         | 833/13931 [00:38<10:55, 19.98it/s]

  6%|▌         | 836/13931 [00:38<11:09, 19.56it/s]

  6%|▌         | 838/13931 [00:38<11:39, 18.71it/s]

  6%|▌         | 840/13931 [00:39<11:44, 18.59

 11%|█         | 1532/13931 [01:13<09:49, 21.05it/s]

 11%|█         | 1535/13931 [01:13<09:08, 22.58it/s]

 11%|█         | 1538/13931 [01:13<08:35, 24.05it/s]

 11%|█         | 1541/13931 [01:13<08:19, 24.81it/s]

 11%|█         | 1544/13931 [01:13<09:31, 21.68it/s]

 11%|█         | 1547/13931 [01:13<08:57, 23.04it/s]

 11%|█         | 1550/13931 [01:14<09:52, 20.88it/s]

 11%|█         | 1553/13931 [01:14<10:13, 20.19it/s]

 11%|█         | 1556/13931 [01:14<10:17, 20.04it/s]

 11%|█         | 1559/13931 [01:14<10:22, 19.88it/s]

 11%|█         | 1562/13931 [01:14<10:27, 19.71it/s]

 11%|█         | 1565/13931 [01:14<09:35, 21.50it/s]

 11%|█▏        | 1568/13931 [01:15<10:05, 20.41it/s]

 11%|█▏        | 1571/13931 [01:15<10:19, 19.94it/s]

 11%|█▏        | 1574/13931 [01:15<09:30, 21.65it/s]

 11%|█▏        | 1577/13931 [01:15<09:57, 20.68it/s]

 11%|█▏        | 1580/13931 [01:15<10:23, 19.81it/s]

 11%|█▏        | 1583/13931 [01:15<10:34, 19.45it/s]

 11%|█▏        | 1586/13931 

 17%|█▋        | 2375/13931 [01:52<15:24, 12.50it/s]

 17%|█▋        | 2379/13931 [01:52<12:14, 15.72it/s]

 17%|█▋        | 2382/13931 [01:52<13:36, 14.14it/s]

 17%|█▋        | 2384/13931 [01:53<16:12, 11.87it/s]

 17%|█▋        | 2387/13931 [01:53<14:14, 13.51it/s]

 17%|█▋        | 2389/13931 [01:53<14:02, 13.69it/s]

 17%|█▋        | 2391/13931 [01:53<14:27, 13.30it/s]

 17%|█▋        | 2393/13931 [01:53<17:36, 10.92it/s]

 17%|█▋        | 2395/13931 [01:54<17:47, 10.81it/s]

 17%|█▋        | 2397/13931 [01:54<20:43,  9.27it/s]

 17%|█▋        | 2399/13931 [01:54<18:01, 10.66it/s]

 17%|█▋        | 2401/13931 [01:54<17:31, 10.97it/s]

 17%|█▋        | 2404/13931 [01:54<15:02, 12.78it/s]

 17%|█▋        | 2406/13931 [01:54<14:41, 13.07it/s]

 17%|█▋        | 2408/13931 [01:55<14:35, 13.17it/s]

 17%|█▋        | 2410/13931 [01:55<14:37, 13.13it/s]

 17%|█▋        | 2412/13931 [01:55<15:30, 12.38it/s]

 17%|█▋        | 2414/13931 [01:55<15:29, 12.39it/s]

 17%|█▋        | 2416/13931 

 21%|██▏       | 2969/13931 [02:34<17:35, 10.38it/s]

 21%|██▏       | 2972/13931 [02:34<15:19, 11.92it/s]

 21%|██▏       | 2974/13931 [02:34<14:16, 12.79it/s]

 21%|██▏       | 2976/13931 [02:34<13:21, 13.67it/s]

 21%|██▏       | 2978/13931 [02:34<12:47, 14.28it/s]

 21%|██▏       | 2980/13931 [02:34<12:20, 14.80it/s]

 21%|██▏       | 2983/13931 [02:34<11:13, 16.26it/s]

 21%|██▏       | 2985/13931 [02:34<11:21, 16.05it/s]

 21%|██▏       | 2987/13931 [02:35<11:56, 15.27it/s]

 21%|██▏       | 2989/13931 [02:35<11:57, 15.24it/s]

 21%|██▏       | 2991/13931 [02:35<12:59, 14.03it/s]

 21%|██▏       | 2993/13931 [02:35<12:41, 14.36it/s]

 21%|██▏       | 2995/13931 [02:35<12:06, 15.06it/s]

 22%|██▏       | 2997/13931 [02:35<11:40, 15.60it/s]

 22%|██▏       | 2999/13931 [02:35<11:26, 15.92it/s]

 22%|██▏       | 3001/13931 [02:36<13:36, 13.38it/s]

 22%|██▏       | 3003/13931 [02:36<12:59, 14.02it/s]

 22%|██▏       | 3006/13931 [02:36<11:45, 15.48it/s]

 22%|██▏       | 3008/13931 

...

 74%|███████▍  | 10356/13931 [08:31<02:14, 26.65it/s]

 74%|███████▍  | 10359/13931 [08:31<02:30, 23.77it/s]

 74%|███████▍  | 10362/13931 [08:31<02:38, 22.48it/s]

 74%|███████▍  | 10365/13931 [08:31<02:45, 21.56it/s]

 74%|███████▍  | 10368/13931 [08:31<02:50, 20.86it/s]

 74%|███████▍  | 10371/13931 [08:32<02:37, 22.67it/s]

 74%|███████▍  | 10374/13931 [08:32<02:25, 24.45it/s]

 74%|███████▍  | 10377/13931 [08:32<02:22, 24.89it/s]

 75%|███████▍  | 10381/13931 [08:32<02:19, 25.50it/s]

 75%|███████▍  | 10384/13931 [08:32<02:14, 26.41it/s]

 75%|███████▍  | 10387/13931 [08:32<02:11, 27.01it/s]

 75%|███████▍  | 10390/13931 [08:32<02:24, 24.59it/s]

 75%|███████▍  | 10393/13931 [08:32<02:18, 25.63it/s]

 75%|███████▍  | 10396/13931 [08:32<02:13, 26.46it/s]

 75%|███████▍  | 10401/13931 [08:33<01:56, 30.37it/s]

 75%|███████▍  | 10405/13931 [08:33<02:18, 25.53it/s]

 75%|███████▍  | 10408/13931 [08:33<02:24, 24.31it/s]

 75%|███████▍  | 10411/13931 [08:33<02:17, 25.52it/s]

 75%|█████

 81%|████████  | 11258/13931 [09:07<01:58, 22.50it/s]

 81%|████████  | 11261/13931 [09:07<02:04, 21.51it/s]

 81%|████████  | 11264/13931 [09:07<02:08, 20.70it/s]

 81%|████████  | 11267/13931 [09:07<02:15, 19.73it/s]

 81%|████████  | 11270/13931 [09:07<02:01, 21.82it/s]

 81%|████████  | 11273/13931 [09:08<02:07, 20.88it/s]

 81%|████████  | 11276/13931 [09:08<02:11, 20.13it/s]

 81%|████████  | 11280/13931 [09:08<01:53, 23.38it/s]

 81%|████████  | 11283/13931 [09:08<01:59, 22.18it/s]

 81%|████████  | 11286/13931 [09:08<02:04, 21.20it/s]

 81%|████████  | 11289/13931 [09:08<02:05, 20.99it/s]

 81%|████████  | 11292/13931 [09:09<02:08, 20.56it/s]

 81%|████████  | 11295/13931 [09:09<02:09, 20.30it/s]

 81%|████████  | 11298/13931 [09:09<02:11, 20.04it/s]

 81%|████████  | 11301/13931 [09:09<02:17, 19.10it/s]

 81%|████████  | 11305/13931 [09:09<01:59, 21.94it/s]

 81%|████████  | 11308/13931 [09:09<02:07, 20.65it/s]

 81%|████████  | 11311/13931 [09:09<02:09, 20.16it/s]

 81%|█████

 86%|████████▋ | 12041/13931 [09:44<01:33, 20.11it/s]

 86%|████████▋ | 12044/13931 [09:44<01:38, 19.09it/s]

 86%|████████▋ | 12046/13931 [09:44<01:41, 18.50it/s]

 86%|████████▋ | 12048/13931 [09:44<01:54, 16.50it/s]

 87%|████████▋ | 12052/13931 [09:44<01:43, 18.11it/s]

 87%|████████▋ | 12054/13931 [09:45<01:56, 16.16it/s]

 87%|████████▋ | 12056/13931 [09:45<01:53, 16.54it/s]

 87%|████████▋ | 12058/13931 [09:45<01:51, 16.84it/s]

 87%|████████▋ | 12060/13931 [09:45<01:47, 17.42it/s]

 87%|████████▋ | 12062/13931 [09:45<01:53, 16.52it/s]

 87%|████████▋ | 12064/13931 [09:45<01:54, 16.33it/s]

 87%|████████▋ | 12066/13931 [09:45<01:56, 16.01it/s]

 87%|████████▋ | 12068/13931 [09:45<01:54, 16.21it/s]

 87%|████████▋ | 12070/13931 [09:46<01:50, 16.82it/s]

 87%|████████▋ | 12072/13931 [09:46<01:50, 16.87it/s]

 87%|████████▋ | 12074/13931 [09:46<01:47, 17.21it/s]

 87%|████████▋ | 12076/13931 [09:46<01:46, 17.36it/s]

 87%|████████▋ | 12078/13931 [09:46<01:48, 17.15it/s]

 87%|█████

 91%|█████████▏| 12729/13931 [10:20<01:02, 19.33it/s]

 91%|█████████▏| 12732/13931 [10:20<01:00, 19.84it/s]

 91%|█████████▏| 12735/13931 [10:20<01:05, 18.39it/s]

 91%|█████████▏| 12740/13931 [10:20<00:54, 21.73it/s]

 91%|█████████▏| 12743/13931 [10:20<00:53, 22.13it/s]

 91%|█████████▏| 12746/13931 [10:21<00:51, 23.21it/s]

 92%|█████████▏| 12749/13931 [10:21<00:58, 20.24it/s]

 92%|█████████▏| 12752/13931 [10:21<01:01, 19.25it/s]

 92%|█████████▏| 12755/13931 [10:21<00:55, 21.34it/s]

 92%|█████████▏| 12758/13931 [10:21<00:55, 21.13it/s]

 92%|█████████▏| 12761/13931 [10:21<00:57, 20.48it/s]

 92%|█████████▏| 12764/13931 [10:21<00:58, 20.04it/s]

 92%|█████████▏| 12767/13931 [10:22<00:57, 20.26it/s]

 92%|█████████▏| 12770/13931 [10:22<00:58, 19.92it/s]

 92%|█████████▏| 12773/13931 [10:22<01:06, 17.36it/s]

 92%|█████████▏| 12775/13931 [10:22<01:06, 17.34it/s]

 92%|█████████▏| 12778/13931 [10:22<01:02, 18.59it/s]

 92%|█████████▏| 12780/13931 [10:22<01:06, 17.28it/s]

 92%|█████

 96%|█████████▋| 13419/13931 [10:56<00:30, 16.53it/s]

 96%|█████████▋| 13421/13931 [10:56<00:32, 15.63it/s]

 96%|█████████▋| 13423/13931 [10:56<00:32, 15.77it/s]

 96%|█████████▋| 13425/13931 [10:56<00:30, 16.57it/s]

 96%|█████████▋| 13427/13931 [10:57<00:29, 16.85it/s]

 96%|█████████▋| 13429/13931 [10:57<00:29, 17.16it/s]

 96%|█████████▋| 13431/13931 [10:57<00:28, 17.64it/s]

 96%|█████████▋| 13433/13931 [10:57<00:28, 17.51it/s]

 96%|█████████▋| 13435/13931 [10:57<00:28, 17.63it/s]

 96%|█████████▋| 13437/13931 [10:57<00:27, 17.84it/s]

 96%|█████████▋| 13440/13931 [10:57<00:24, 19.82it/s]

 96%|█████████▋| 13443/13931 [10:57<00:25, 18.80it/s]

 97%|█████████▋| 13445/13931 [10:58<00:27, 17.94it/s]

 97%|█████████▋| 13447/13931 [10:58<00:26, 18.01it/s]

 97%|█████████▋| 13450/13931 [10:58<00:25, 19.07it/s]

 97%|█████████▋| 13452/13931 [10:58<00:28, 16.73it/s]

 97%|█████████▋| 13454/13931 [10:58<00:31, 15.33it/s]

 97%|█████████▋| 13456/13931 [10:58<00:31, 14.91it/s]

 97%|█████

In [351]:
X = np.array(Img)

In [352]:
X.shape

(11870, 256, 256, 13)

In [378]:
X_piece = X[0:300,:,:,:]
X_piece = X_piece.astype(float)

In [371]:
y_piece = category[0:300]

d1={x: x-y_piece.min() for x in y_piece}
key_left = set(range(0,32)) - set(d1.keys())
value_left = set(range(0,32)) - set(d1.values())
d2 = {list(key_left)[i]:list(value_left)[i] for i in range(len(key_left))}
d2.update(d1)
def dict20(a):
    return np.array([d2[k] for k in a])

y_piece = dict20(y_piece)

In [382]:
X_piece.shape,len(y_piece)

(torch.Size([300, 256, 256, 13]), 300)

In [379]:
X_piece = torch.from_numpy(X_piece)
y_piece = torch.from_numpy(y_piece)

The size we want is (300,13,256,256) rather than (300,256,256,13)

In [432]:
t = np.array(X_piece)

In [483]:
X_converted = np.ndarray(shape=(300,13,256,256), dtype=float, order='F')

In [484]:
def change(arr):
    y=np.array([arr[:,:,0],arr[:,:,1],arr[:,:,2],arr[:,:,3],arr[:,:,4],arr[:,:,5],arr[:,:,6],arr[:,:,7],arr[:,:,8],
            arr[:,:,9],arr[:,:,10],arr[:,:,11],arr[:,:,12]])
    return y

In [485]:
for i in range(300):
    X_converted[0,:,:,:] = change(t[0,:,:,:])

In [486]:
X_converted = X_converted.astype(float)
X_converted = torch.from_numpy(X_converted)

In [509]:
d3 = {7:2,8:1,0:0}
y_piece = np.array([d3[k] for k in np.array(y_piece)])

In [514]:
y_piece = torch.from_numpy(y_piece)

In [520]:
np.save('/Volumes/ksankaran/ImageNet//Python Data/X_converted',np.array(X_converted))
np.save('/Volumes/ksankaran/ImageNet//Python Data/y_piece',np.array(y_piece))

In [515]:
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(13, 64, kernel_size=(5,5), stride=1, padding=2),
            BatchNorm2d(64),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=(3,3), stride=(2,2),padding = 1),
            
            Conv2d(64, 128, kernel_size=(5,5), stride=1, padding=2),
            BatchNorm2d(128),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=(3,3), stride=2, padding = 1),
            
            Conv2d(128, 256, kernel_size=(5,5), stride=1, padding=2),
            BatchNorm2d(256),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=(3,3), stride=2, padding = 1),
            
            Conv2d(256, 512, kernel_size=(5,5), stride=1, padding=2),
            BatchNorm2d(512),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=(3,3), stride=2, padding = 1),
            
            AdaptiveAvgPool2d((1, 1))
        )
        self.linear_layers = Sequential(
            Linear(in_features= 512, out_features=8, bias = True)
        )
        


    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [516]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.0001)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(13, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=1, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=(3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=(3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
    (12): Conv2d(256, 512, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (13): BatchNorm2d(512, eps=1e-

In [517]:
model = model.double()

In [518]:
# batch size of the model
batch_size = 8

# number of epochs to train the model
n_epochs = 5

for epoch in range(1, n_epochs+1):
    print("epoch",epoch,'\n')

    # keep track of training and validation loss
    train_loss = 0.0
        
    permutation = torch.randperm(X_converted.size()[0])

    training_loss = []
    for i in tqdm.tqdm(range(0,X_converted.size()[0], batch_size)):

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_converted[indices],y_piece[indices]
        if torch.cuda.is_available():
            batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
        
        optimizer.zero_grad()
        outputs = model(batch_x)
        outputs.squeeze(1)
        loss = criterion(outputs,batch_y)
        training_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        
    training_loss = np.average(training_loss)
    print('epoch: \t', epoch, '\t training loss: \t', training_loss)



  0%|          | 0/38 [00:00<?, ?it/s]

epoch 1 





  3%|▎         | 1/38 [00:24<15:03, 24.43s/it]

  5%|▌         | 2/38 [00:46<14:17, 23.81s/it]

  8%|▊         | 3/38 [01:09<13:36, 23.34s/it]

 11%|█         | 4/38 [01:30<12:56, 22.84s/it]

 13%|█▎        | 5/38 [01:51<12:12, 22.21s/it]

 16%|█▌        | 6/38 [02:12<11:38, 21.82s/it]

 18%|█▊        | 7/38 [02:35<11:24, 22.07s/it]

 21%|██        | 8/38 [02:56<10:55, 21.86s/it]

 24%|██▎       | 9/38 [03:17<10:27, 21.62s/it]

 26%|██▋       | 10/38 [03:38<09:58, 21.36s/it]

 29%|██▉       | 11/38 [03:58<09:29, 21.09s/it]

 32%|███▏      | 12/38 [04:22<09:28, 21.86s/it]

 34%|███▍      | 13/38 [04:45<09:13, 22.12s/it]

 37%|███▋      | 14/38 [05:08<09:01, 22.54s/it]

 39%|███▉      | 15/38 [05:31<08:41, 22.69s/it]

 42%|████▏     | 16/38 [05:54<08:18, 22.66s/it]

 45%|████▍     | 17/38 [06:16<07:53, 22.56s/it]

 47%|████▋     | 18/38 [06:39<07:31, 22.58s/it]

 50%|█████     | 19/38 [07:01<07:08, 22.54s/it]

 53%|█████▎    | 20/38 [07:24<06:46, 22.60s/it]

 55%|█████▌    | 21/38 [07:

epoch: 	 1 	 training loss: 	 1.0801677408971695
epoch 2 





  3%|▎         | 1/38 [00:20<12:41, 20.58s/it]

  5%|▌         | 2/38 [00:43<12:43, 21.21s/it]

  8%|▊         | 3/38 [01:06<12:41, 21.74s/it]

 11%|█         | 4/38 [01:29<12:33, 22.16s/it]

 13%|█▎        | 5/38 [01:51<12:06, 22.01s/it]

 16%|█▌        | 6/38 [02:12<11:36, 21.77s/it]

 18%|█▊        | 7/38 [02:33<11:11, 21.65s/it]

 21%|██        | 8/38 [02:54<10:42, 21.41s/it]

 24%|██▎       | 9/38 [03:15<10:18, 21.31s/it]

 26%|██▋       | 10/38 [03:36<09:56, 21.31s/it]

 29%|██▉       | 11/38 [03:58<09:37, 21.40s/it]

 32%|███▏      | 12/38 [04:20<09:21, 21.59s/it]

 34%|███▍      | 13/38 [04:43<09:07, 21.89s/it]

 37%|███▋      | 14/38 [05:05<08:48, 22.01s/it]

 39%|███▉      | 15/38 [05:28<08:31, 22.23s/it]

 42%|████▏     | 16/38 [05:49<08:01, 21.90s/it]

 45%|████▍     | 17/38 [06:10<07:34, 21.66s/it]

 47%|████▋     | 18/38 [06:31<07:12, 21.61s/it]

 50%|█████     | 19/38 [06:53<06:52, 21.71s/it]

 53%|█████▎    | 20/38 [07:16<06:33, 21.89s/it]

 55%|█████▌    | 21/38 [07:

...

epoch: 	 3 	 training loss: 	 0.39395365850888237
epoch 4 





  3%|▎         | 1/38 [00:20<12:47, 20.75s/it]

  5%|▌         | 2/38 [00:43<12:44, 21.24s/it]

  8%|▊         | 3/38 [01:06<12:43, 21.83s/it]

 11%|█         | 4/38 [01:29<12:31, 22.10s/it]

 13%|█▎        | 5/38 [01:51<12:14, 22.27s/it]

 16%|█▌        | 6/38 [02:14<11:59, 22.50s/it]

 18%|█▊        | 7/38 [02:37<11:42, 22.66s/it]

 21%|██        | 8/38 [03:00<11:24, 22.81s/it]

 24%|██▎       | 9/38 [03:24<11:04, 22.90s/it]

 26%|██▋       | 10/38 [03:46<10:37, 22.76s/it]

 29%|██▉       | 11/38 [04:07<10:03, 22.37s/it]

 32%|███▏      | 12/38 [04:30<09:39, 22.28s/it]

 34%|███▍      | 13/38 [04:53<09:27, 22.68s/it]

 37%|███▋      | 14/38 [05:17<09:12, 23.04s/it]

 39%|███▉      | 15/38 [05:40<08:49, 23.03s/it]

 42%|████▏     | 16/38 [06:01<08:15, 22.54s/it]

 45%|████▍     | 17/38 [06:24<07:52, 22.51s/it]

 47%|████▋     | 18/38 [06:46<07:25, 22.28s/it]

 50%|█████     | 19/38 [07:08<07:06, 22.44s/it]

 53%|█████▎    | 20/38 [07:31<06:42, 22.36s/it]

 55%|█████▌    | 21/38 [07:

epoch: 	 4 	 training loss: 	 0.40505495917600676
epoch 5 





  3%|▎         | 1/38 [00:21<13:26, 21.80s/it]

  5%|▌         | 2/38 [00:44<13:12, 22.01s/it]

  8%|▊         | 3/38 [01:06<12:54, 22.13s/it]

 11%|█         | 4/38 [01:28<12:32, 22.12s/it]

 13%|█▎        | 5/38 [01:50<12:01, 21.86s/it]

 16%|█▌        | 6/38 [02:11<11:37, 21.79s/it]

 18%|█▊        | 7/38 [02:34<11:27, 22.18s/it]

 21%|██        | 8/38 [02:58<11:18, 22.61s/it]

 24%|██▎       | 9/38 [03:21<11:01, 22.83s/it]

 26%|██▋       | 10/38 [03:43<10:28, 22.44s/it]

 29%|██▉       | 11/38 [04:04<09:57, 22.14s/it]

 32%|███▏      | 12/38 [04:27<09:42, 22.41s/it]

 34%|███▍      | 13/38 [04:50<09:24, 22.59s/it]

 37%|███▋      | 14/38 [05:13<09:00, 22.51s/it]

 39%|███▉      | 15/38 [05:36<08:43, 22.78s/it]

 42%|████▏     | 16/38 [06:00<08:26, 23.01s/it]

 45%|████▍     | 17/38 [06:22<07:59, 22.84s/it]

 47%|████▋     | 18/38 [06:44<07:33, 22.65s/it]

 50%|█████     | 19/38 [07:07<07:08, 22.55s/it]

 53%|█████▎    | 20/38 [07:29<06:45, 22.52s/it]

 55%|█████▌    | 21/38 [07:

epoch: 	 5 	 training loss: 	 0.38562003970666847


In [519]:
#torch.save(model,'/Volumes/ksankaran/ImageNet/Models/sub_mod1')

The model has been pre-trained.

In [521]:
X_target_train= np.load('/Volumes/ksankaran/sen12ms_subsample2/X_target_train.npy')
X_target_test  = np.load('/Volumes/ksankaran/sen12ms_subsample2/X_target_test.npy')
y_target_test = np.load('/Volumes/ksankaran/sen12ms_subsample2/y_target_test.npy')
y_target_train = np.load('/Volumes/ksankaran/sen12ms_subsample2/y_target_train.npy')

In [522]:
X_source = np.load('/Volumes/ksankaran/sen12ms_subsample2/X_source.npy')
y_source = np.load('/Volumes/ksankaran/sen12ms_subsample2/y_source.npy')
X_source.shape,y_source.shape

((300, 13, 256, 256), (300,))

In [524]:
y_target_test = y_target_test-1
y_target_train = y_target_train -1
y_source = y_source -1

In [525]:
d1={x: x-y_source.min() for x in y_source}
key_left = set(range(0,10)) - set(d1.keys())
value_left = set(range(0,10)) - set(d1.values())
d2 = {list(key_left)[i]:list(value_left)[i] for i in range(len(key_left))}
d2.update(d1)

In [523]:
import geotransfer
from geotransfer.model_pre_functions import *

In [526]:
def prepare_data(x,y):
    y = np.array([d2[k] for k in y])
    y = torch.from_numpy(y)
    
    x = x.astype(float)
    x = torch.from_numpy(np.array(x,dtype='f'))
    return x,y

In [527]:
train_x,train_y = prepare_data(X_source,y_source)
train_x.shape,train_y.shape

(torch.Size([300, 13, 256, 256]), torch.Size([300]))

In [536]:
def train_net(train_x,train_y,model,criterion, optimizer,
              batch_size=8,n_epochs=2):

    for epoch in range(1, n_epochs+1):
        print("epoch",epoch,'\n')
    
        # keep track of training and validation loss
        train_loss = 0.0
        
        permutation = torch.randperm(train_x.size()[0])

        training_loss = []
        for i in tqdm(range(0,train_x.size()[0], batch_size)):

            indices = permutation[i:i+batch_size]
            batch_x, batch_y = train_x[indices],train_y[indices]
            if torch.cuda.is_available():
                batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
        
            optimizer.zero_grad()
            outputs = model(batch_x)
            outputs.squeeze(1)
            loss = criterion(outputs,batch_y)
            training_loss.append(loss.item())
            loss.backward()
            optimizer.step()
            
        training_loss = np.average(training_loss)
        print('epoch: \t', epoch, '\t training loss: \t', training_loss)

In [530]:
model2 = Net()
model2 = model2.double()
model2.load_state_dict(model.state_dict())
model2.linear_layers = Sequential(
            Linear(in_features= 512, out_features=10, bias = True)
        )

In [531]:
for param in model2.cnn_layers.parameters():
    param.requires_grad = False
for parameter in model2.linear_layers.parameters():
    parameter.requires_grad = True
    

In [533]:
optimizer = Adam(model2.linear_layers.parameters(), lr=0.0001)
# defining the loss function
criterion = CrossEntropyLoss()

In [534]:
model2.float()

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(13, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=1, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=(3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=(3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
    (12): Conv2d(256, 512, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (13): BatchNorm2d(512, eps=1e-

In [537]:
train_net(train_x,train_y,model2,criterion,optimizer)



  0%|          | 0/38 [00:00<?, ?it/s]

epoch 1 





  3%|▎         | 1/38 [00:03<02:08,  3.46s/it]

  5%|▌         | 2/38 [00:06<02:02,  3.39s/it]

  8%|▊         | 3/38 [00:10<02:01,  3.47s/it]

 11%|█         | 4/38 [00:13<01:57,  3.45s/it]

 13%|█▎        | 5/38 [00:17<01:52,  3.42s/it]

 16%|█▌        | 6/38 [00:20<01:48,  3.41s/it]

 18%|█▊        | 7/38 [00:23<01:43,  3.33s/it]

 21%|██        | 8/38 [00:26<01:38,  3.27s/it]

 24%|██▎       | 9/38 [00:29<01:34,  3.25s/it]

 26%|██▋       | 10/38 [00:33<01:29,  3.21s/it]

 29%|██▉       | 11/38 [00:36<01:25,  3.15s/it]

 32%|███▏      | 12/38 [00:39<01:21,  3.14s/it]

 34%|███▍      | 13/38 [00:42<01:17,  3.10s/it]

 37%|███▋      | 14/38 [00:45<01:13,  3.07s/it]

 39%|███▉      | 15/38 [00:48<01:10,  3.07s/it]

 42%|████▏     | 16/38 [00:51<01:07,  3.05s/it]

 45%|████▍     | 17/38 [00:54<01:03,  3.04s/it]

 47%|████▋     | 18/38 [00:57<01:00,  3.05s/it]

 50%|█████     | 19/38 [01:00<00:57,  3.05s/it]

 53%|█████▎    | 20/38 [01:03<00:54,  3.05s/it]

 55%|█████▌    | 21/38 [01:

epoch: 	 1 	 training loss: 	 1.6298792017133612
epoch 2 





  3%|▎         | 1/38 [00:03<01:53,  3.06s/it]

  5%|▌         | 2/38 [00:06<01:49,  3.04s/it]

  8%|▊         | 3/38 [00:09<01:46,  3.05s/it]

 11%|█         | 4/38 [00:12<01:43,  3.04s/it]

 13%|█▎        | 5/38 [00:15<01:39,  3.02s/it]

 16%|█▌        | 6/38 [00:18<01:36,  3.02s/it]

 18%|█▊        | 7/38 [00:21<01:33,  3.01s/it]

 21%|██        | 8/38 [00:24<01:30,  3.01s/it]

 24%|██▎       | 9/38 [00:27<01:27,  3.01s/it]

 26%|██▋       | 10/38 [00:30<01:24,  3.00s/it]

 29%|██▉       | 11/38 [00:33<01:20,  3.00s/it]

 32%|███▏      | 12/38 [00:36<01:17,  3.00s/it]

 34%|███▍      | 13/38 [00:39<01:14,  3.00s/it]

 37%|███▋      | 14/38 [00:42<01:11,  2.99s/it]

 39%|███▉      | 15/38 [00:45<01:08,  2.99s/it]

 42%|████▏     | 16/38 [00:48<01:05,  2.99s/it]

 45%|████▍     | 17/38 [00:51<01:02,  2.99s/it]

 47%|████▋     | 18/38 [00:54<00:59,  3.00s/it]

 50%|█████     | 19/38 [00:57<00:57,  3.01s/it]

 53%|█████▎    | 20/38 [01:00<00:54,  3.03s/it]

 55%|█████▌    | 21/38 [01:

epoch: 	 2 	 training loss: 	 1.3420366472319554
